# Locating Public Recycle Bin Near You
Run the following cells to find the nearest public recycle bin near you in NYC. 

This notebook creates 2 outputs:
1) table containing information on finding a recycle bin near you 

2) an interractive map showing all the recycle bins in nyc and your location

## Download Data, Libraries & Your Current Location


In [ ]:
!pip install folium 
!pip install geocoder
!pip install geopy


In [1]:
import folium
import geocoder
import geopy
from geopy import distance
import pandas as pd
import numpy as np


### Load your current location 

In [2]:
g= geocoder.ip('me') #This finds your current location using IP

In [3]:
my_location=g.latlng
my_location

[40.7484, -73.9967]

###  Load data 
The dataset is taken from NYC OpenData

In [4]:
url= 'https://data.cityofnewyork.us/resource/sxx4-xhzg.csv'
data = pd.read_csv(url)
data.head()

,borough,site_type,park_site_name,address,latitude,longitude
0,Bronx,Subproperty,227th St. Plgd,E 227 St/Bronx River Pkway,40.890849,-73.864224
1,Bronx,Subproperty,Allerton Ballfields,Allerton Ave & Moshulu Pkway,40.848891,-73.877128
2,Bronx,Outdoor,Arthur Ave & E 187 St,Arthur Ave & 187 St,40.855570,-73.887565
3,Bronx,Outdoor,Barstow Mansion,"895 Shore Road, Pelham Bay Park",40.871864,-73.805549
4,Bronx,Subproperty,Bradley Playground,2001-2017 Bronx Park E,40.851889,-73.868549


## Data Cleaning & Preparation

In [5]:
data.count()

borough           545
site_type         545
park_site_name    545
address           540
latitude          544
longitude         544
dtype: int64

##### Drops the empty columns that do not have location information

In [6]:
data=data.dropna(subset=['latitude','longitude'])
data.count()

borough           544
site_type         544
park_site_name    544
address           539
latitude          544
longitude         544
dtype: int64

##### combines the latitude and longitude columns together 

In [7]:
data['geo']=list(zip(data.latitude,data.longitude))
data['geo']

0        (40.890848989, -73.864223918)
1      (40.8488907878, -73.8771283938)
2       (40.85557, -73.88756499999998)
3              (40.871864, -73.805549)
4              (40.851889, -73.868549)
                    ...               
540            (40.637121, -74.119287)
541            (40.603832, -74.158697)
542             (40.603828, -74.16125)
543            (40.517368, -74.190913)
544    (40.518652, -74.18730999999998)
Name: geo, Length: 544, dtype: object

#### Calculates the distance between your location and the recycle bins locations

In [10]:
l=[]
for index, row in data.iterrows():
    
    miles=distance.distance(my_location, row['geo']).miles
    l.append(miles)
    

data['distance']=l



## The 10 nearest recycle bin near you within 1 mile

In [11]:
x=1  #distance in miles
m=[]
a_values=[]
type_values=[]
for index, row in data.iterrows():
    
    miles=distance.distance(my_location, row['geo']).miles
    if miles<x:
        m.append(miles)
        address= row['address']
        ty=row['site_type']
        a_values.append(address)
        type_values.append(ty)
        

output=pd.DataFrame({'address':a_values, 'type':type_values, 'miles': m})
        
output.sort_values(by=['miles'])[:10]


,address,type,miles
4,Chelsea Park,Outdoor,0.239921
5,430 W 25 St,Indoor,0.278515
0,34 St & 7 Ave,Outdoor,0.352490
12,W 20 St,Outdoor,0.519908
7,Hudson River Park,Outdoor,0.543223
6,Hudson River Park,Outdoor,0.562756
14,Pier 62 Garden Area,Outdoor,0.636590
8,Broadway Btwn 39 St & 40 St,Outdoor,0.638717
11,High Line Park,Outdoor,0.646571
15,Pier 62 Skate Park,Outdoor,0.660365


## Visualization
The interactive map shows your location(red) and all the recycle bins location in nyc. Clicking on the markers shows addition information. 

In [12]:
# creates a empty map
my_map = folium.Map(location=(40.730610,-73.935242), zoom_start=12)


In [13]:
#Visualize the geospatial data in the csv file 

for index, row in data.iterrows():
    
        folium.Marker((row['latitude'], row['longitude']),popup='<b>Address:</b> '+str(row['address'])+' <b>Type:</b> '+str(row['site_type']) , tooltip=row['park_site_name']).add_to(my_map)
        
folium.Marker(my_location, tooltip='<b>Im here<b>',icon=folium.Icon(color="red")).add_to(my_map) 
display(my_map)



### References: 

https://geocoder.readthedocs.io/api.html#install
https://www.dataquest.io/blog/jupyter-notebook-tutorial/
https://python-visualization.github.io/folium/quickstart.html#Getting-Started
https://geopy.readthedocs.io/en/stable/#module-geopy.distance
https://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html
https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/
https://python-visualization.github.io/folium/quickstart.html#Getting-Started